## Understanding Directory naming [WSL][#WSL][copilot]  

Directory from terminal
** # explorer.exe . ** 

### Your terminal’s way of saying:

> **“You’re user `bhuns`, logged into a machine called `Puget1`, currently sitting in your home directory (`~`).”**

### What Each Part Means

| Terminal Segment       | Meaning                                                                 |
|------------------------|-------------------------------------------------------------------------|
| `bhuns`                | Your **Linux username** inside WSL (Windows Subsystem for Linux)        |
| `@Puget1`              | The **hostname** of your machine (likely your Windows PC’s name)        |
| `:~`                   | You're in your home directory (`~` = `/home/bhuns`)                     |
| `$`                    | The **shell prompt** symbol for a regular (non-root) user               |

---

### 🧠 Why It Says That

- WSL sets your **user** and **computer name** into the prompt, similar to Linux and macOS terminals.
- The terminal prompt is defined by an environment variable called `$PS1`.

For example, in your case it’s probably set to something like:

Which expands to:
- `\u` → current user: `bhuns`
- `\h` → hostname: `Puget1`
- `\w` → current working directory
- `\$` → prompt character (`$` or `#` if you're root)

### 🎨 Want to Customize It?

You can personalize the prompt to include Git branch info, colors, or even emoji. Just edit your `~/.bashrc`:

```bash
nano ~/.bashrc
```

Look for the line that sets `PS1` and tweak away—or I can whip up a smart version that even shows your active Conda environment and Git branch. Want that kind of tricked-out prompt?

# Visual-style cheat sheet


**WSL (Linux)** and **Windows** filesystems connect &
How to navigate between them easily:
---

##3 🧭 WSL ↔ Windows File System Navigation

| **Action** | **In WSL (Ubuntu Bash)** | **In Windows (Explorer)** | **What It Does** |
|------------|---------------------------|----------------------------|------------------|
| Access **C:\Users** | `/mnt/c/Users/` | `C:\Users\` | Lets WSL read/write to your Windows files |
| Go to WSL **Home (`~`)** | `cd ~` or `cd /home/bhuns` | `\\wsl$\Ubuntu\home\bhuns` | Access your WSL/Linux files from Windows |
| Open current folder in Explorer | `explorer.exe .` | N/A | Launches Windows Explorer in current WSL dir |
| Launch Notepad from WSL | `notepad.exe filename.txt` | N/A | Opens a file using Windows Notepad |
| Map WSL folder as drive | (N/A) | Use `Map Network Drive` to `\\wsl$\Ubuntu\...` | Mount Linux files as a Windows drive |

> 🧠 Replace `Ubuntu` with your actual distro name if needed, like `Ubuntu-22.04`

---

### 🗂️ Folder Mapping Examples

| **Linux Path**               | **Windows View**                          |
|-----------------------------|-------------------------------------------|
| `/home/bhuns/JL_1`          | `\\wsl$\Ubuntu\home\bhuns\JL_1`          |
| `/mnt/c/Users/bhuns/Desktop`| `C:\Users\bhuns\Desktop`                  |
| `/mnt/d/Projects`           | `D:\Projects`                             |

---

Would you like a printable PDF or PNG version of this for your desktop or workspace? I can draw it up in seconds as a quick-reference chart.

## Where to launch (using WSL) [copilot]

### 🗂️ Where to Run Your Commands

Here’s the rundown of where each type of command should live:

| **Command**                              | **Run From**              | **Why**                                                                 |
|------------------------------------------|----------------------------|--------------------------------------------------------------------------|
| `conda activate scu`                     | Anywhere                   | Activates your environment globally; doesn’t depend on your folder.     |
| `jupyter lab`                            | **From your project dir** (e.g. `~/scu-project`) | So JupyterLab opens with your notebooks right there.                   |
| `git add`, `commit`, `push`, etc.        | **From inside your repo** (e.g. `~/scu-project`) | Git actions only apply to the repo you're inside.                      |

---

### ✅ Your Ideal Workflow (All Together)

```bash
cd ~/scu-project         # 1. Go to your project folder
conda activate scu       # 2. Activate your Conda environment
jupyter lab              # 3. Start JupyterLab from inside the project
```

> This ensures your notebooks open from the correct location, your Git status reflects the right repo, and your environment is loaded properly.

# Loading and Writing Tables (JL to/from XL) [Preseving formulas][copilot]

Ah, gotcha—this is a key distinction, Bill. When pandas loads data from Excel via `read_excel`, it pulls in **evaluated values only**. Formulas in cells get flattened into the result they produce, not the underlying equation. To preserve formulas through the entire process, we’ll need to level up with `openpyxl`, which gives us direct access to the worksheet’s structure—including cell formulas.

---

## 🧪 Updated Strategy for Preserving Formulas

We’ll upgrade your workflow like this:

### 1. **Load Excel with Formulas Intact**  
Instead of relying on `pandas.read_excel`, we’ll use `openpyxl` to:
- Read cell values **and formulas**
- Build a DataFrame that holds formulas where applicable

### 2. **Write Back While Keeping Formulas**  
When writing, we’ll:
- Loop through the DataFrame
- Preserve formulas by re-inserting them into the Excel cells, rather than overwriting with static values

---

## 📦 Enhanced `load_df` Function (Preserve Formulas)

```python
from openpyxl import load_workbook
import pandas as pd

def load_df_with_formulas(file_name="xl_tbl.xlsx", sheet_name="xl_tbl"):
    wb = load_workbook(file_name, data_only=False)  # `data_only=False` preserves formulas
    ws = wb[sheet_name]

    data = []
    for row in ws.iter_rows(values_only=False):
        row_data = []
        for cell in row:
            if cell.value is not None:
                if cell.data_type == 'f':  # It's a formula
                    row_data.append(f"={cell.value}")
                else:
                    row_data.append(cell.value)
            else:
                row_data.append(None)
        data.append(row_data)

    # Extract headers from first row
    headers = data[0]
    df = pd.DataFrame(data[1:], columns=headers)
    print(f"✅ Loaded sheet '{sheet_name}' from '{file_name}', preserving formulas.")
    return df
```

---

## 📤 Enhanced `write_df` Function (Write Back Formulas)

```python
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

def write_df_with_formulas(df, file_name="xl_tbl.xlsx", sheet_name="xl_tbl", table_name="xl_tbl"):
    wb = Workbook()
    ws = wb.active
    ws.title = sheet_name

    # Write headers
    for col_idx, col_name in enumerate(df.columns, start=1):
        ws.cell(row=1, column=col_idx, value=col_name)

    # Write values or formulas
    for row_idx, row in enumerate(df.itertuples(index=False), start=2):
        for col_idx, value in enumerate(row, start=1):
            cell = ws.cell(row=row_idx, column=col_idx)
            if isinstance(value, str) and value.startswith("="):
                cell.value = value[1:]  # Remove leading '=' for Excel formula
                cell.data_type = 'f'
            else:
                cell.value = value

    # Define table
    nrows, ncols = df.shape
    table_range = f"A1:{get_column_letter(ncols)}{nrows+1}"

    from openpyxl.worksheet.table import Table, TableStyleInfo
    table = Table(displayName=table_name, ref=table_range)
    style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True)
    table.tableStyleInfo = style
    ws.add_table(table)

    wb.save(file_name)
    print(f"💾 Wrote updated DataFrame to Excel file '{file_name}', preserving formulas.")
```

---

These enhanced functions will respect your formula logic and keep the round-trip editing seamless. Would you like me to wrap both into a module like `xl_formula_sync.py` with built-in checks for formula integrity? You could use it across any number of `_tbl` projects without reinventing the wheel.